In [ ]:
api_key = "b550c0f1-670c-48dd-aba9-81e75cdd43b7"

In [ ]:
import requests
import pandas as pd
import time

def fetch_all_data(url, headers):
    all_data = []
    next_cursor = None
    pages_fetched = 0
    queries = 0

    while True:

        if next_cursor is not None:
            paginated_url = f"{url}&start_date=1946-10-24&end_date=2023-06-12&cursor={next_cursor}"
        else:
            paginated_url = f"{url}&start_date=1946-10-24&end_date=2023-06-12"


        response = requests.get(paginated_url, headers=headers)
        queries += 1
        data = response.json()


        if not data['data']:
            break


        all_data.extend(data['data'])


        if 'meta' in data and 'next_cursor' in data['meta']:
            next_cursor = data['meta']['next_cursor']
        else:
            break


        if queries >= 29:
            time.sleep(60)
            queries = 0


        pages_fetched += 1

    return all_data


url = "https://api.balldontlie.io/v1/stats?seasons[]=2023"
headers = {
    "Authorization": "b550c0f1-670c-48dd-aba9-81e75cdd43b7"
}


data = fetch_all_data(url, headers)

df = pd.json_normalize(data)


df


""


In [ ]:
import datetime

def fetch_data_for_months(url, headers, start_date, end_date):
    all_data = []
    current_date = start_date

    while current_date <= end_date:
        start_of_month = current_date.replace(day=1)
        end_of_month = start_of_month.replace(day=28) + datetime.timedelta(days=4)
        end_of_month = end_of_month - datetime.timedelta(days=end_of_month.day)

        month_data = fetch_all_data(url, headers, start_of_month, end_of_month)
        all_data.extend(month_data)

        current_date = end_of_month + datetime.timedelta(days=1)

    return all_data

url = "https://api.balldontlie.io/v1/stats?seasons[]=2023"
headers = {
    "Authorization": "b550c0f1-670c-48dd-aba9-81e75cdd43b7"
}


start_date = datetime.datetime(2023, 10, 1)
end_date = datetime.datetime(2024, 2, 29)

data = fetch_data_for_months(url, headers, start_date, end_date)


df = pd.json_normalize(data)

print(df.head())


TypeError: fetch_all_data() takes 2 positional arguments but 4 were given

In [ ]:
df.columns


Index(['id', 'min', 'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm',
       'fta', 'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'turnover',
       'pf', 'pts', 'player.id', 'player.first_name', 'player.last_name',
       'player.position', 'player.height', 'player.weight',
       'player.jersey_number', 'player.college', 'player.country',
       'player.draft_year', 'player.draft_round', 'player.draft_number',
       'player.team_id', 'team.id', 'team.conference', 'team.division',
       'team.city', 'team.name', 'team.full_name', 'team.abbreviation',
       'game.id', 'game.date', 'game.season', 'game.status', 'game.period',
       'game.time', 'game.postseason', 'game.home_team_score',
       'game.visitor_team_score', 'game.home_team_id', 'game.visitor_team_id'],
      dtype='object')

In [ ]:
df

,id,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,...,game.date,game.season,game.status,game.period,game.time,game.postseason,game.home_team_score,game.visitor_team_score,game.home_team_id,game.visitor_team_id
0,13932867,30,5,13,0.384615,2,9,0.222222,0,0,...,2023-10-24,2023,Final,4,Final,False,119,107,8,14
1,13932868,35,7,11,0.636364,1,2,0.500000,0,0,...,2023-10-24,2023,Final,4,Final,False,119,107,8,14
2,13932869,36,12,22,0.545455,3,5,0.600000,2,4,...,2023-10-24,2023,Final,4,Final,False,119,107,8,14
3,13932870,36,8,12,0.666667,2,3,0.666667,2,2,...,2023-10-24,2023,Final,4,Final,False,119,107,8,14
4,13932871,34,8,13,0.615385,3,5,0.600000,2,2,...,2023-10-24,2023,Final,4,Final,False,119,107,8,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,13942214,00,0,0,0.000000,0,0,0.000000,0,0,...,2023-10-26,2023,Final,4,Final,False,118,117,17,23
496,13942215,17,2,3,0.666667,0,1,0.000000,0,0,...,2023-10-26,2023,Final,4,Final,False,118,117,17,23
497,13942216,20,2,6,0.333333,1,4,0.250000,2,2,...,2023-10-26,2023,Final,4,Final,False,118,117,17,23
498,13942217,27,3,4,0.750000,1,1,1.000000,2,2,...,2023-10-26,2023,Final,4,Final,False,118,117,17,23


###2024

In [ ]:
import requests
import pandas as pd
import time

def fetch_all_data(url, headers, start_date, end_date):
    all_data = []
    next_cursor = None
    pages_fetched = 0
    queries = 0

    while True:
        if next_cursor is not None:
            paginated_url = f"{url}&start_date={start_date}&end_date={end_date}&cursor={next_cursor}"
        else:
            paginated_url = f"{url}&start_date={start_date}&end_date={end_date}"

        response = requests.get(paginated_url, headers=headers)
        queries += 1
        data = response.json()

        if not data['data']:
            break

        all_data.extend(data['data'])

        if 'meta' in data and 'next_cursor' in data['meta']:
            next_cursor = data['meta']['next_cursor']
        else:
            break

        if queries >= 590:
            time.sleep(60)
            queries = 0

        pages_fetched += 1

    return all_data

base_url = "https://api.balldontlie.io/v1/stats?seasons[]="
headers = {
    "Authorization": "b550c0f1-670c-48dd-aba9-81e75cdd43b7"
}

start_date = "2023-10-24"
end_date = "2024-03-09"

for year in range(2023, 2024):
    url = base_url + str(year)
    data = fetch_all_data(url, headers, start_date, end_date)
    df = pd.json_normalize(data)
    print(f"Fetched data for season {year}")


Fetched data for season 2023


In [ ]:
df

,id,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,...,game.date,game.season,game.status,game.period,game.time,game.postseason,game.home_team_score,game.visitor_team_score,game.home_team_id,game.visitor_team_id
0,13932867,30,5,13,0.384615,2,9,0.222222,0,0,...,2023-10-24,2023,Final,4,Final,False,119,107,8,14
1,13932868,35,7,11,0.636364,1,2,0.500000,0,0,...,2023-10-24,2023,Final,4,Final,False,119,107,8,14
2,13932869,36,12,22,0.545455,3,5,0.600000,2,4,...,2023-10-24,2023,Final,4,Final,False,119,107,8,14
3,13932870,36,8,12,0.666667,2,3,0.666667,2,2,...,2023-10-24,2023,Final,4,Final,False,119,107,8,14
4,13932871,34,8,13,0.615385,3,5,0.600000,2,2,...,2023-10-24,2023,Final,4,Final,False,119,107,8,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33485,14508661,00,0,0,0.000000,0,0,0.000000,0,0,...,2024-03-09,2023,Final,5,Final,False,128,118,25,28
33486,14508662,00,0,0,0.000000,0,0,0.000000,0,0,...,2024-03-09,2023,Final,5,Final,False,128,118,25,28
33487,14508663,00,0,0,0.000000,0,0,0.000000,0,0,...,2024-03-09,2023,Final,5,Final,False,128,118,25,28
33488,14508664,00,0,0,0.000000,0,0,0.000000,0,0,...,2024-03-09,2023,Final,5,Final,False,128,118,25,28


In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
df.to_csv('Current_Year.csv', index=False)

!cp Current_Year.csv "drive/My Drive/"